In [141]:
import numpy as np
import pandas as pd
import sys, os
import re 
import csv
import shlex
import json
import datetime


In [142]:
logfile = '/home/tech/haproxy_log_analysis/haproxy.log.'

In [ ]:
with open(logfile) as file:
    lines = file.read().splitlines()

In [56]:
pattern_json = re.compile(r'{.*}')
pattern_select_words = re.compile(r'[^:,{}]+')

pattern_method = re.compile(r'(^[A-Z]+\b)')
pattern_url = re.compile(r'([/bh].*)\s')
pattern_protocol = re.compile(r'(\s[H].*)')
content = []

        

In [57]:
for line in lines:
    line = str(line)
    for match in re.findall(pattern_json, line):
        string_json = match
    string_json = re.sub(pattern_select_words, r'"\g<0>"', string_json)
    json_data = json.loads(string_json)
    
    data = []
    
    timestamp = int(json_data.get('timestamp'))
    data.append(datetime.datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d'))
    data.append(datetime.datetime.fromtimestamp(timestamp).strftime('%H:%M:%S'))
    
    http_request = str(json_data.get('http_request',None))
    for url in re.findall(pattern_url, http_request):
        data.append(url)
    for method in re.findall(pattern_method, http_request):
        data.append(method)
    for protocol in re.findall(pattern_protocol, http_request):
        data.append(protocol)
    
    data.append(json_data.get('upstream_addr'))
    data.append(json_data.get('remote_addr'))
    data.append(int(json_data.get('http_status')))
    data.append(int(json_data.get('session_duration')))
    data.append(int(json_data.get('upstream_connect_time')))
    data.append(int(json_data.get('upstream_response_time')))
    
    content.append(data) 
    

In [3]:
columns = ['day', 'time', 'url', 'method', 'protocol', 'upstream_addr',
           'remote_addr', 'http_status', 'session_duration',
           'upstream_connect_time', 'upstream_response_time']

df = pd.DataFrame(content, columns=columns)
#df.to_csv('/home/tech/haproxy_log_analysis/haproxy.csv', index = None)


NameError: name 'pd' is not defined

In [96]:
new = df.groupby(['remote_addr','upstream_addr','method'])['upstream_response_time'].agg(['sum','count'])
n=new.reset_index()
n1=n.groupby(['remote_addr','upstream_addr','method']).sum()
n1.rename(columns={'count':'hits','sum':'UPST_RT_duration'},inplace=True)

In [97]:
a=n1.reset_index()
a['UPST_RT_min']=(a.UPST_RT_duration)/(60)
a['UPST_RT_TOTAL'] = pd.to_timedelta((60*a['UPST_RT_min']).astype('int'), unit='s')
a1 = a.sort_values(by=['UPST_RT_TOTAL'], ascending=False)
a1 = a1.head(100)
del a1['UPST_RT_duration']
del a1['UPST_RT_min']
a1['UPST_RT_avg']=(a1.UPST_RT_TOTAL)/(a1.hits)
a1 = a1.reset_index()
del a1['index']

In [2]:
l = []
l = a1.remote_addr
sd = []
p = []
for i in l:
    a = df[df.remote_addr == i]
    print(type(a))
    x = np.std(a['upstream_response_time'])
    sd.append(x)
    p1 = np.percentile(a['upstream_response_time'],90)
    p.append(p1)
sd_array = pd.Series(sd)
p_array = pd.Series(p)
a1['SD'] = sd_array.values
a1['90_percentile'] = p_array.values
a1.to_csv('/home/tech/haproxy_log_analysis/haproxy_log_report.csv', index = None)

NameError: name 'a1' is not defined

NameError: name 'a' is not defined